# Libraries for this research notebook

In [1]:
import pandas as pd
import os

# to overcome path issue for src
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import sys

# set the path to the current file
current_file_path = Path().resolve()

# set the path to the src folder
src_folder_path = current_file_path.parent

# add the src folder to the system path
sys.path.append(str(src_folder_path))

import src.data_loader as DB

# Query data from MySQL

In [ ]:
# instantiate the DataLoader object
load_data = DB.DataLoader()

# create a connection
conn = load_data.initiate_local_connection()

In [ ]:
# load in data set using string query
query = '''
    SELECT *
    FROM time_series.oil
'''

results = load_data.query_from_string(conn, query)

results

In [ ]:
# load in data set using .sql file
query_file_path = '../src/scripts/query_data.sql'

results = load_data.query_from_file(conn, query_file_path)

results